In [2]:
import numpy as np
from pyxem.signals.diffraction_simulation import DiffractionSimulation

from pyxem.utils.sim_utils import get_electron_wavelength,\
    get_kinematical_intensities
import pymatgen as pmg

In [4]:
E = pmg.Element("Pt")
cubic_lattice = pmg.Lattice.cubic(5)
structure = pmg.Structure.from_spacegroup(195,cubic_lattice, [E], [[0, 0, 0]])

In [18]:
Ga = pmg.Element("Ga")
As = pmg.Element("As")
lattice = pmg.Lattice.hexagonal(3.91,6.44)
u = (1/8)
structure = pmg.Structure.from_spacegroup(168,lattice, [Ga,Ga,As,As], [[0, 0, 0],[2/3,1/3,1/2],[0,0,0.5+u],[2/3,1/3,u]])

In [19]:
# Specify variables used in calculation
wavelength = 1
max_excitation_error = 1
debye_waller_factors = {}
latt = structure.lattice
reciprocal_radius = 1

# Obtain crystallographic reciprocal lattice points within `max_r` and
# g-vector magnitudes for intensity calculations.
recip_latt = latt.reciprocal_lattice_crystallographic
recip_pts, g_hkls = \
       recip_latt.get_points_in_sphere([[0, 0, 0]], [0, 0, 0],
                                       reciprocal_radius,
                                       zip_results=False)[:2]
cartesian_coordinates = recip_latt.get_cartesian_coords(recip_pts)

# Identify points intersecting the Ewald sphere within maximum
# excitation error and store the magnitude of their excitation error.
radius = 1 / wavelength
r = np.sqrt(np.sum(np.square(cartesian_coordinates[:, :2]), axis=1))
theta = np.arcsin(r / radius)
z_sphere = radius * (1 - np.cos(theta))
proximity = np.absolute(z_sphere - cartesian_coordinates[:, 2])
intersection = proximity < max_excitation_error
# Mask parameters corresponding to excited reflections.
intersection_coordinates = cartesian_coordinates[intersection]
intersection_indices = recip_pts[intersection]
proximity = proximity[intersection]
g_hkls = g_hkls[intersection]
#Calculate diffracted intensities based on a kinematical model.
intensities = get_kinematical_intensities(structure,
                                          intersection_indices,
                                          g_hkls,
                                          proximity,
                                          max_excitation_error,
                                          debye_waller_factors)

#Threshold peaks included in simulation based on minimum intensity.
peak_mask = intensities > 1e-20
intensities = intensities[peak_mask]
intersection_coordinates = intersection_coordinates#[peak_mask]
intersection_indices = intersection_indices#[peak_mask]

In [21]:
intersection_indices

array([[-3.,  0., -2.],
       [-3.,  0., -1.],
       [-3.,  0.,  0.],
       ..., 
       [ 3.,  0.,  0.],
       [ 3.,  0.,  1.],
       [ 3.,  0.,  2.]])